In [1]:
import RPi.GPIO as GPIO
from time import sleep

In [2]:
# GPIO 핀 번호 설정
IN1 = 17
IN2 = 27
ENA = 18
IN3 = 5
IN4 = 6
ENB = 13
STBY = 25

In [3]:
# GPIO 초기 설정 함수
def setup_gpio():
    # BCM 모드 설정
    GPIO.setmode(GPIO.BCM)
    # 모터 제어핀 출력 모드로 설정
    GPIO.setup(IN1, GPIO.OUT)
    GPIO.setup(IN2, GPIO.OUT)
    GPIO.setup(ENA, GPIO.OUT)
    GPIO.setup(IN3, GPIO.OUT)
    GPIO.setup(IN4, GPIO.OUT)
    GPIO.setup(ENB, GPIO.OUT)
    GPIO.setup(STBY, GPIO.OUT)

In [4]:
# PWM 초기화 및 시작
def start_pwm():
    global pwmA, pwmB
    # A와 B 모터용 PWM을 1000Hz로 설정
    pwmA = GPIO.PWM(ENA, 1000)
    pwmB = GPIO.PWM(ENB, 1000)
    pwmA.start(0)  # 초기 듀티 사이클을 0으로 시작
    pwmB.start(0)

In [5]:
# 모터 방향 설정 함수
def set_motor(channel, state):
    if channel == 'A':
        GPIO.output(IN1, state[0])
        GPIO.output(IN2, state[1])
    elif channel == 'B':
        GPIO.output(IN3, state[0])
        GPIO.output(IN4, state[1])

In [6]:
# 모터 전진
def motor_forward(channel, speed):
    set_motor(channel, (GPIO.HIGH, GPIO.LOW))  # 정방향 회전을 위해 설정
    if channel == 'A':
        pwmA.ChangeDutyCycle(speed)  # A 모터 속도 설정
    elif channel == 'B':
        pwmB.ChangeDutyCycle(speed)  # B 모터 속도 설정

In [7]:
# 모터 후진
def motor_backward(channel, speed):
    set_motor(channel, (GPIO.LOW, GPIO.HIGH))  # 역방향 회전을 위해 설정
    if channel == 'A':
        pwmA.ChangeDutyCycle(speed)  # A 모터 속도 설정
    elif channel == 'B':
        pwmB.ChangeDutyCycle(speed)  # B 모터 속도 설정

In [8]:
# 모터 제어 루프 함수
def motor_control_loop():
    for _ in range(2):
        # 모터 드라이버 활성화
        GPIO.output(STBY, GPIO.HIGH)
        motor_forward('A', 50)  # A 모터 전진
        motor_forward('B', 50)  # B 모터 전진
        sleep(1)  # 1초 대기
        motor_backward('A', 50)  # A 모터 후진
        motor_backward('B', 50)  # B 모터 후진
        sleep(1)  # 1초 대기

In [9]:
# 정리 함수
def cleanup():
    #global pwmA, pwmB
    pwmA.stop()  # A 모터 PWM 정지
    pwmB.stop()  # B 모터 PWM 정지
    GPIO.cleanup()  # GPIO 핀 해제

In [10]:
setup_gpio()          # GPIO 초기 설정
start_pwm()           # PWM 시작
motor_control_loop()  # 모터 제어 루프 실행
cleanup()             # 종료 후 정리